In [1]:

import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,773 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 htt

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import avg, year, round
from pyspark.sql.types import DecimalType

average_prices_bdrm = df.filter(df['bedrooms'] == 4).groupBy(year(df['date']).alias('year')).agg(avg('price').alias('average_price'))
average_prices_bdrm = average_prices_bdrm .withColumn("average_price", round(average_prices_bdrm ["average_price"].cast(DecimalType(10, 2)), 2))
average_prices_bdrm.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2019|    300263.70|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_prices_bdrm_bath = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3)).groupBy(year(df['date']).alias('year')).agg(avg('price').alias('average_price'))
average_prices_bdrm_bath = average_prices_bdrm_bath.withColumn("average_price", round(average_prices_bdrm_bath["average_price"].cast(DecimalType(10, 2)), 2))
average_prices_bdrm_bath.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    292725.69|
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
+----+-------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
from pyspark.sql.functions import avg, year

average_prices_year_bdrm_bath_floor_size = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_living'] >= 2000)).groupBy(year(df['date']).alias('year')).agg(avg('price').alias('average_price'))
average_prices_year_bdrm_bath_floor_size = average_prices_year_bdrm_bath_floor_size.withColumn("average_price", round(average_prices_year_bdrm_bath_floor_size["average_price"], 2))
average_prices_year_bdrm_bath_floor_size.show()



+----+-------------+
|year|average_price|
+----+-------------+
|2022|    290242.99|
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
+----+-------------+



In [8]:
from pyspark.sql.functions import avg
import time

# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

average_price_greater_than_350k = df.filter(df['price'] >= 350000).agg(avg('price').alias('average_price')).collect()[0][0]
average_price_greater_than_350k = "{:.2f}".format(average_price_greater_than_350k)

view_rating = df.filter(df['price'] >= 350000).select('view').distinct().collect()[0]['view']

print("Average Price: $" + str(average_price_greater_than_350k))
print("View Rating: " + view_rating)


print("--- %s seconds ---" % (time.time() - start_time))

Average Price: $473796.26
View Rating: 7
--- 1.9265201091766357 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView("home_sales")
spark.table("home_sales").cache()


DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

import time

# Query with cached data
start_time_cached = time.time()
result_cached = spark.table("home_sales").filter(df['price'] >= 350000).select('view').distinct().collect()
runtime_cached = time.time() - start_time_cached

# Query with uncached data
start_time_uncached = time.time()
result_uncached = df.filter(df['price'] >= 350000).select('view').distinct().collect()
runtime_uncached = time.time() - start_time_uncached

# Print results and runtime comparison
print("Results with cached data:")
for row in result_cached:
    print(row)

print("\nRuntime with cached data: %.2f seconds" % runtime_cached)

print("\nResults with uncached data:")
for row in result_uncached:
    print(row)

print("\nRuntime with uncached data: %.2f seconds" % runtime_uncached)



Results with cached data:
Row(view='7')
Row(view='51')
Row(view='15')
Row(view='54')
Row(view='11')
Row(view='29')
Row(view='69')
Row(view='42')
Row(view='87')
Row(view='73')
Row(view='64')
Row(view='3')
Row(view='30')
Row(view='34')
Row(view='59')
Row(view='8')
Row(view='28')
Row(view='22')
Row(view='85')
Row(view='16')
Row(view='35')
Row(view='52')
Row(view='0')
Row(view='71')
Row(view='98')
Row(view='99')
Row(view='47')
Row(view='96')
Row(view='43')
Row(view='5')
Row(view='31')
Row(view='100')
Row(view='18')
Row(view='70')
Row(view='27')
Row(view='61')
Row(view='75')
Row(view='17')
Row(view='26')
Row(view='46')
Row(view='78')
Row(view='89')
Row(view='77')
Row(view='6')
Row(view='90')
Row(view='68')
Row(view='60')
Row(view='19')
Row(view='23')
Row(view='41')
Row(view='55')
Row(view='93')
Row(view='95')
Row(view='38')
Row(view='40')
Row(view='25')
Row(view='44')
Row(view='82')
Row(view='53')
Row(view='92')
Row(view='86')
Row(view='58')
Row(view='81')
Row(view='33')
Row(view='48')
Row(

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("parquet_home_sales", mode="overwrite")


In [21]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet("parquet_home_sales")


In [22]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('parquet_home_sales')

In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

from pyspark.sql.functions import avg

start_time = time.time()

filtered_df = p_df.filter(p_df['view'] >= 350000)
average_price = filtered_df.agg(avg('price')).collect()[0][0]

if average_price is not None:
    print("Average Price:", round(average_price, 2))
else:
    print("No data available")

print("--- %s seconds ---" % (time.time() - start_time))

No data available
--- 0.5780954360961914 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('parquet_home_sales')

In [28]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('parquet_home_sales')
print(is_cached)


False
